In [2]:
# %%
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
import IPython
from stochastic.processes.continuous import FractionalBrownianMotion
def box_containment_code_function(one_dim_coord, one_dim_box_size):
    
    half_box_size = one_dim_box_size / 2 #assuming origin is in the middle
    
    if one_dim_coord > half_box_size:
        #outside box in +ve half or coord system
        new_one_dim_coord = box_containment_algorithm(one_dim_coord, one_dim_box_size, 0)
        

    elif one_dim_coord < -half_box_size:
        #outside box in +ve half or coord system
        new_one_dim_coord = box_containment_algorithm(one_dim_coord, one_dim_box_size, 1)
    
    return new_one_dim_coord
def box_containment_checker(initial_fbm_coord_array, box_size_in_x, box_size_in_y, box_size_in_z):#can also be used for cuboids
    
    #x_coord, y_coord, z_coord = *initial_fbm_coord_array
    box_dimensions = np.array([])
    box_dimensions = np.append(box_dimensions, box_size_in_x)
    box_dimensions = np.append(box_dimensions, box_size_in_y)
    box_dimensions = np.append(box_dimensions, box_size_in_z)
    
    one_dim_coord_with_1d_box_dimension_pair = np.column_stack((initial_fbm_coord_array, box_dimensions))
    
    new_array = np.array([])
    
    """if x_coord > (box_size_in_x / 2):
        new_x_coord = cube_containment_algorithm(x_coord, box_size_in_x, 0)
        new_array = np.append(new_array, new_x_coord)


    elif x_coord < (-box_size_in_x / 2):
        new_x_coord = cube_containment_algorithm(x_coord, box_size_in_x, 1)
        new_array = np.append(new_array, new_x_coord)
    
    elif (-box_size_in_x / 2) <= x_coord <= (box_size_in_x / 2):
        new_array = np.append(new_array, x_coord)"""
    
    
    for coord_and_dimension_pair in one_dim_coord_with_1d_box_dimension_pair:
        coord, dimension = coord_and_dimension_pair
        
        if coord <= (dimension/2):
            new_array = np.append(new_array, coord)
        elif coord >= (-dimension/2):
            new_array = np.append(new_array, coord)
        else:
            new_array = np.append(new_array, box_containment_code_function(coord, dimension))
        
        
    return new_array
    
def box_containment_algorithm(one_dim_coordinate_outside_box, one_dim_box_size, positive_or_negative_overshoot): #+ve = 0, -ve = 1
    distance_of_boundary_overshoot = (-1 ** positive_or_negative_overshoot) * one_dim_coordinate_outside_box - one_dim_box_size 
    #right as move_to not move_by - markdown explanation

    #number_of_reflections_counter = positive_or_negative_overshoot
    while distance_of_boundary_overshoot > (one_dim_box_size / 2):
        distance_of_boundary_overshoot = distance_of_boundary_overshoot - (one_dim_box_size / 2)
        #lhs is actually new coord now
        
        positive_or_negative_overshoot += 1

    one_dim_coordinate_outside_box = distance_of_boundary_overshoot * -1 * positive_or_negative_overshoot
    
    return one_dim_coordinate_outside_box
class MyAgent(ap.Agent):
    
    def setup(self):
        
        all_samples_from_fgn_fbm = FractionalBrownianMotion(t=self.p.steps, hurst=self.p.hurst)
        all_samples_from_fgn_array = all_samples_from_fgn_fbm.sample(n=self.p.steps * 3 * self.p.fps)
        x_samples_from_fgn_array = np.array([])
        self.x=np.insert(x_samples_from_fgn_array, 0, all_samples_from_fgn_array[:self.p.steps * self.p.fps])
        y_samples_from_fgn_array = np.array([])
        self.y=np.insert(y_samples_from_fgn_array, 0, all_samples_from_fgn_array[self.p.steps * self.p.fps:
                                                                                 self.p.steps*2 * self.p.fps])
        z_samples_from_fgn_array = np.array([])
        self.z=np.insert(z_samples_from_fgn_array, 0, all_samples_from_fgn_array[self.p.steps*2 * self.p.fps:])        
    
    def setup_space(self, space):
        
        self.space = space
        self.in_contact = 0 #0 for not in contact and 1 for in contact
        self.step_number = 0 #each agent carries the step number so they know which position to move to
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
        
    def update_step_number(self):
        self.step_number += 1
                    
                
    def update_position(self):

        temp_array = np.array([])
            
        
        temp_array = np.append(temp_array, self.x[self.step_number])
        temp_array = np.append(temp_array, self.y[self.step_number])
        temp_array = np.append(temp_array, self.z[self.step_number])
        
        normalisation = np.linalg.norm(temp_array) * self.p.fps
        
        if normalisation != 0:
            normalised_temp_array = temp_array / normalisation
            self.space.move_by(self, normalised_temp_array)
        
        else:
            self.space.move_by(self, temp_array)
    
        
    def update_position_with_collisions(self):
        
        temp_array = np.array([])
        temp_array = np.append(temp_array, self.x[self.step_number])
        temp_array = np.append(temp_array, self.y[self.step_number])
        temp_array = np.append(temp_array, self.z[self.step_number])

        normalisation = np.linalg.norm(temp_array) * self.p.fps #step size reflects the number of samples per second
        if normalisation != 0:
            normalised_temp_array = temp_array / normalisation
            self.space.move_by(self, normalised_temp_array)
            if len(self.neighbors(self, distance=1))>=1:#reflect the original agent's path causing the collision:
                self.collision_position = self.pos #record position of collision for heat map
                
                self.x[self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] = self.x[
                    self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] * -1
                
                self.y[self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] = self.y[
                    self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] * -1
                
                self.z[self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] = self.z[
                    self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] * -1
                
                for neighbour in self.neighbors(self, distance=self.p.collision_distance):#there is a collision
                    #reflect neighbour intermediate steps until the next step generated from FGN
                    neighbour.x[neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] = neighbour.x[
                        neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] * -1
                    
                    neighbour.y[neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] = neighbour.y[
                        neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] * -1
                    
                    neighbour.z[neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] = neighbour.z[
                        neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] * -1
                

        else:#if normalisation=0 then still take collisions into account
            self.space.move_by(self, temp_array)
            if len(self.neighbors(self, distance=1))>=1:#reflect the original agent's path causing the collision:
                self.collision_position = self.pos #record position of collision for heat map
                
                self.x[self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] = self.x[
                    self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] * -1
                
                self.y[self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] = self.y[
                    self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] * -1
                
                self.z[self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] = self.z[
                    self.step_number*self.p.fps:(self.step_number+1)*self.p.fps] * -1
                
                for neighbour in self.neighbors(self, distance=1):#there is a collision
                    #reflect neighbour intermediate steps until the next step generated from FGN
                    neighbour.x[neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] = neighbour.x[
                        neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] * -1
                    
                    neighbour.y[neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] = neighbour.y[
                        neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] * -1
                    
                    neighbour.z[neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] = neighbour.z[
                        neighbour.step_number*self.p.fps:(neighbour.step_number+1)*self.p.fps] * -1

        
class MyModel(ap.Model):
    
    def setup(self):
        self.space = ap.Space(self, shape=[self.p.size_x]+[self.p.size_y]+[self.p.size_z], torus=True)
        self.agents = ap.AgentList(self, self.p.population, MyAgent)
        self.space.add_agents(self.agents, random=True) #agents start in random positions
        self.agents.setup_space(self.space)
        
        
    def step(self):
        if self.p.collisions==1:#take collisions into account
            counter = 0
            while counter < self.p.fps:
                self.agents.update_position_with_collisions()
                self.record('Collision position', self.agents.collision_position)
                counter+=1
        else:
            self.agents.update_position()
        
        self.agents.update_step_number()


def animation_plot_single(m, ax):
    ndim = 3
    ax.set_title(f"FBM Model {ndim}D t={m.t}")
    pos = m.space.positions.values()
    pos = np.array(list(pos)).T  # Transform (Transpose) positions of agents
    ax.scatter(*pos, s=1, c='black')
    ax.set_xlim(0, m.p.size_x)
    ax.set_ylim(0, m.p.size_y)
    ax.set_zlim(0, m.p.size_z)
    ax.set_axis_off()

def animation_plot(m, p):
    projection = '3d'
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111, projection=projection)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=10))

parameters = {
    'hurst': 0.5,
    'size_x': 1,
    'size_y': 1,
    'size_z': 1,
    'steps': 200, #this is equivalent to the time
    'population': 5,
    'collisions': 1,
    'fps': 20,
    'collision_distance': 1 #distance away agents must be to collide
}
animation_plot(MyModel, parameters)
#you will see particles overlapping with as they collide and then rebound off each other but do not go through each other
#or may be passing each other but only aligned in 2d not in the third dimension also
#to create heat map
model = MyModel(parameters)
results = model.run()
results.variables.MyModel.head()
import pandas as pd
#proving there are collisions taking place
df = results.variables.MyModel
print(df["Collision position"].value_counts())



Completed: 200 steps
Run time: 0:00:10.060502
Simulation finished
[[0.8771815738123145, 0.3001205977102008, 0.8699722009103209], [0.08203750366570425, 0.6084731250584631, 0.8269996716345015], [0.12862419384362103, 0.8330545434347755, 0.263808445578369], [0.8040456184040172, 0.7395414021249186, 0.39725319185649605], [0.5695508058709828, 0.47974015186537977, 0.09614628106518838]]    200
Name: Collision position, dtype: int64
